<a href="https://colab.research.google.com/github/Nomi-lpr/skills-communicate-using-markdown/blob/main/happy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip mcp-tool-api-main.zip

Archive:  mcp-tool-api-main.zip
ba5d5d88f7bab6f7f6d749a14d9633636c3fef94
   creating: mcp-tool-api-main/
  inflating: mcp-tool-api-main/.gitignore  
   creating: mcp-tool-api-main/configs/
  inflating: mcp-tool-api-main/configs/both_tools_config.json  
  inflating: mcp-tool-api-main/configs/crop_only_config.json  
 extracting: mcp-tool-api-main/configs/no_tools_config.json  
  inflating: mcp-tool-api-main/configs/video_only_config.json  
  inflating: mcp-tool-api-main/data_postprocess.py  
  inflating: mcp-tool-api-main/main.py  
  inflating: mcp-tool-api-main/mcp_api_usage.py  
  inflating: mcp-tool-api-main/mcp_utils.py  
  inflating: mcp-tool-api-main/recheck_score.py  
  inflating: mcp-tool-api-main/tool_test.py  
   creating: mcp-tool-api-main/tools/
 extracting: mcp-tool-api-main/tools/__init__.py  
  inflating: mcp-tool-api-main/tools/crop_server.py  
  inflating: mcp-tool-api-main/tools/grounding_server.py  
  inflating: mcp-tool-api-main/tools/video_server.py  
   creating: mc

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
! ls



drive  mcp-tool-api-main  mcp-tool-api-main.zip  sample_data


In [6]:
!pip install openai httpx numpy decord Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 90.6 MB/s eta 0:00:00


In [7]:
%cd /content/mcp-tool-api-main/

/content/mcp-tool-api-main


In [10]:
!pip install mcp==1.9.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00


In [11]:
import asyncio
import json
import os
from mcp_api_usage import MCPClient
from mcp_utils import format_conversation

In [12]:
VIDEO_DIR = "./my_videos/"

In [13]:
# --- 将这里的值替换成您从火山引擎获取的真实凭证 ---

# 1. 替换成您的火山引擎API Key
os.environ["OPENAI_API_KEY"] = "f387cf21-352d-4486-a88c-b017147acacd"  # <--- 替换


# 2. 设置火山引擎的Base URL (注意：末尾的斜杠要去掉，因为openai库会自动拼接)
os.environ["OPENAI_BASE_URL"] = "https://ark.cn-beijing.volces.com/api/v3" # <--- 替换

# 3. 替换成您想使用的火山引擎模型的ID
os.environ["OPENAI_MODEL"] = "Doubao-Seed-1.6-thinking"  # <--- 替换


In [15]:
# 3. 原始的 load_data 和 edit_data 函数可以保留或删除，因为我们不再调用它们

def edit_data(data, tool_name):
    for item in data:
        item["question"] = f"Use the tool {tool_name} to answer the question: {item['question']}"
    return data

def load_data(data, tool_name):
    if tool_name=="crop":
        data=[{
            "id": 5123,
            "dataset": "arkitscenes",
            "scene_name": "47331668",
            "question_type": "crop",
            "question": "How many people appears in the poster titled blackmail in this room? Use the tool crop to answer the question. You should pick the related image to pass back to the tool.",
            "ground_truth": "D",
            "options": [
              "A. 1",
              "B. 2",
              "C. 3",
              "D. 4"
            ],
            "video_path": "arkitscenes/47331668.mp4"
        }]
    elif tool_name=="extract_video_segment":
        data=[{
            "id": 5123,
            "dataset": "arkitscenes",
            "scene_name": "47331668",
            "question": f"You MUST use the tool extract_video_segment to look closer at the video to answer the following question. You should pick the related moment or frame index to pass back to the tool. {data[0]['question']}",
            "ground_truth": "I dont know either.",
            "video_path": "arkitscenes/47331668.mp4",
        }]
    return data


async def main():
    # --- 使用您自己的数据和Prompt ---

    # a. 定义您的视频文件名
    my_video_filename = "my_test_video.mp4"  # <--- 请确保这个名字和您上传的视频文件名一致

    # b. 【核心】在这里修改和实验您的Prompt！
    my_prompt = "视频里那个快速移动的物体是什么？请使用工具仔细观察。"

    # c. 将您的数据构造成脚本需要的格式
    data = [{
        "id": 9999,  # 可以是任意ID
        "question": my_prompt,
        "video_path": my_video_filename
    }]

    # --- 原有测试逻辑保持不变 ---

    # 初始化MCP client
    mcp_client = MCPClient()
    try:
        await mcp_client.initialize()

        # 遍历数据（我们的例子里只有一条）
        for i, item in enumerate(data):
            print(f"\n=== 正在处理第 {i+1}/{len(data)} 项测试 ===")

            # 为每个测试项创建一个独立的会话
            session = mcp_client.create_session()

            question = item["question"]
            # 使用我们定义的视频路径和问题进行查询
            result = await session.query(
                question,
                video_paths=[os.path.join(VIDEO_DIR, item["video_path"])],
                image_paths=None
            )

            print(f"会话ID: {result['session_id']}")
            print("--- 对话全过程 ---")
            print(format_conversation(result["conversation"], verbose=True))

            # 程序会在这里暂停，让您进入调试模式检查变量
            print("\n--- 程序已暂停，进入调试模式 (Pdb) ---")
            print("您可以输入 'result' 查看返回结果，或输入 'c' 继续运行，输入 'q' 退出。")
            breakpoint()

    finally:
        # 清理所有资源
        await mcp_client.cleanup()
        print("\n--- 测试结束，资源已清理 ---")



In [16]:
if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop